In [1]:
import numpy as np
import pandas as pd
import pprint
pd.options.display.max_columns = 100
idx = pd.IndexSlice

%matplotlib inline
import matplotlib.pyplot as pt
import seaborn as sb
sb.set_style('whitegrid')

import wikifunctions as wf

import requests, json, re, time, os

path = '\data'
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import unquote, quote
from copy import deepcopy
from SPARQLWrapper import SPARQLWrapper, JSON


In [12]:
title="Jeff Bezos"
title=title.replace(" ","%20")
url="https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles="+title+"&format=json"

print(title, url)

Jeff%20Bezos https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles=Jeff%20Bezos&format=json


In [25]:
w=requests.get(url).json()
pprint.pprint(w)

{'batchcomplete': '',
 'query': {'pages': {'142528': {'ns': 0,
                                'pageid': 142528,
                                'pageprops': {'defaultsort': 'Bezos, Jeff',
                                              'page_image_free': 'Jeff_Bezos_at_Amazon_Spheres_Grand_Opening_in_Seattle_-_2018_(39074799225)_(cropped).jpg',
                                              'wikibase-badge-Q17437798': '1',
                                              'wikibase-shortdesc': 'American '
                                                                    'engineer '
                                                                    'and '
                                                                    'entrepreneur, '
                                                                    'founder '
                                                                    'of '
                                                                    'Amazon.com, '
                   

In [28]:
key=list(w['query']['pages'].keys())[0]
z=w['query']['pages'][key]['pageprops']["wikibase_item"]
#print(key)
print(z)

Q312556


In [ ]:
title="Jeff Bezos"
title=title.replace(" ","%20")
url="https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles="+title+"&format=json"
w=requests.get(url).json()
key=list(w['query']['pages'].keys())[0]
df.at[i,'QID']=w['query']['pages'][key]['pageprops']["wikibase_item"]

In [2]:
def writeQuery(year,month):
    query='''PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?entity ?entityLabel ?sexual_orientation ?sexual_orientationLabel ?sex_or_gender ?sex_or_genderLabel ?pronouns ?pronounsLabel ?reference91 ?reference21 ?reference6553 WHERE {
          ?entity wdt:P569 ?dateOfBirth;
                  wdt:P31 wd:Q5. 
          FILTER("'''+str(year)+'''-'''+str(month)+'''-00"^^xsd:dateTime <= ?dateOfBirth &&
             ?dateOfBirth < "'''+str(year)+'''-'''+str(month+1)+'''-00"^^xsd:dateTime) 
          
          OPTIONAL { ?entity wdt:P91 ?sexual_orientation. }
          OPTIONAL { ?entity p:P91 [prov:wasDerivedFrom [ pr:P958 ?reference91 ]]. }
          OPTIONAL { ?entity wdt:P21 ?sex_or_gender. }
          OPTIONAL { ?entity p:P21 [prov:wasDerivedFrom [ pr:P958 ?reference21 ]]. }
          OPTIONAL { ?entity wdt:P6553 ?pronouns. }
          OPTIONAL { ?entity p:P6553 [prov:wasDerivedFrom [ pr:P958 ?reference6553 ]]. }
          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
        }
        ORDER BY (?birth)
        '''
    return query

#FILTER((?birth >= "'''+str(year)+'''-01-01"^^xsd:dateTime) && (?birth <= "'''+str(year)+'''-12-31"^^xsd:dateTime))


def getData(queryString):
    sparql = SPARQLWrapper("https://query.wikidata.org/bigdata/namespace/wdq/sparql")
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        data = sparql.query().convert()
   # ret is a stream with the results in XML, see <http://www.w3.org/TR/rdf-sparql-XMLres/>
    except :
        print('query failed')

    #url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    #data = requests.get(url, params={'query': query}).json()
    return data['results']['bindings']

def queryAsCSV(data):
    #try:
        #data = requests.get(url, params={'query': query, 'format': 'json','encoding':'UTF-8'}).json()
    csv=[]
    for item in data:#['results']['bindings']:


        try:
            entity=item['entity']['value'].strip('http://www.wikidata.org/entity/')
        except:
            entity=''
        try:
            name=item['entityLabel']['value']
        except:
            name=''
        try:
            sexuality=item['sexual_orientationLabel']['value']
        except:
            sexuality=''
        try:
            sexOrGender=item['sex_or_genderLabel']['value']
        except:
            sexOrGender=''
        try:
            pronouns=item['pronounsLabel']['value']
        except:
            pronouns=''
        try:
            ref91=item['reference91']['value']
        except:
            ref91=''
        try:
            ref21=item['reference21']['value']
        except:
            ref21=''
        try:
            ref6553=item['reference6553']['value']
        except:
            ref6553=''
  


        csv.append({
            'QID':entity,
            'Name':name,
            'P21':sexOrGender,
            'P21ref':ref21,
            'P91':sexuality,
            'P91ref':ref91,
            'P6553':pronouns,
            'P6553ref':ref6553
        })



    #except:
        
        #print(str(i) +' '+ str(j))
        #fullYear=0
        
    return csv


In [3]:
df=pd.DataFrame(columns=['QID', 'Name', 'P21', 'P21ref', 'P91', 'P91ref', 'P6553', 'P6553ref'])
for i in range(1950,2021):
    for j in range(1,13):
        try:
            df=df.append(pd.DataFrame(queryAsCSV(getData(writeQuery(i,j)))))
            #query=writeQuery(i)
            #data=getData(query)
            #csv=queryAsCSV(data)
            #d=pd.DataFrame(csv)
            #df=df.append(d)
        except:
            print(str(j)+'/'+str(i%100)+' failed')
    if i%5==0:
        print(str(i))
        df.to_csv('wikidataSupportData.csv',index=False)
df.to_csv('wikidataSupportData.csv',index=False)

1950
1955
1960
1965
1970
1975
1980
1985
1990
1995
2000
2005
query failed
11/8 failed
query failed
8/9 failed
query failed
9/9 failed
2010
query failed
10/11 failed
2015
2020


### Getting references

In [2]:
df=pd.read_csv('mergedPeople.csv',index_col=0, keep_default_na=False)
print(len(df))
df.head()

C:\Users\Jack\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1464111


,Name,Year,inTransWomen,inTransMen,inNB,inTrans,inWomen,inMen,inLGBT,inBisexual_people,inGay_men,inLesbians,inQueer,links,categories,QID,Name_,P21,P21ref,P91,P91ref,P6553,P6553ref,pronounsEn,trans,gay,queer
0,Kjell Aamot,1950.0,False,False,False,False,False,False,False,False,False,False,False,"['BI Norwegian Business School', 'Verdens Gang']","['Category:1950 births', 'Category:BI Norwegia...",Q11980824,Kjell Aamot,male,,,,,,,False,False,False
1,Olav Aaraas,1950.0,False,False,False,False,False,False,False,False,False,False,False,"['Fredrikstad', 'Maihaugen', 'Norwegian Museum...","['Category:1950 births', 'Category:Directors o...",Q3431302,Olav Aaraas,male,,,,,,,False,False,False
2,Marc Aaronson,1950.0,False,False,False,False,False,False,False,False,False,False,False,"['United States', 'Astronomer', 'Los Angeles, ...","['Category:1950 births', 'Category:1987 deaths...",Q2258893,Marc Aaronson,male,,,,,,,False,False,False
3,Ülle Aaskivi,1950.0,False,False,False,False,True,False,False,False,False,False,False,"['1990 Estonian Supreme Soviet election', 'Con...","['Category:1950 births', 'Category:2007 deaths...",Q16407530,Ülle Aaskivi,female,,,,,,,False,False,False
4,Oğuz Abadan,1950.0,False,False,False,False,False,False,False,False,False,False,False,"['Turkey', 'Ankara', 'Mandolin', 'Gazi Univers...","['Category:1950 births', 'Category:Living peop...",Q7116769,Oğuz Abadan,male,,,,,,,False,False,False


In [53]:
def writeRefQuery(QID,p):
    query='''PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?ref WHERE {
        wd:'''+QID+''' p:'''+p+''' ?statement.
        ?statement ps:'''+p+''' ?education.
        OPTIONAL {
        ?statement prov:wasDerivedFrom ?refnode.
        ?refnode pr:P248 ?ref.
        }
        OPTIONAL {
        ?statement prov:wasDerivedFrom ?refnode.
        ?refnode pr:P854 ?ref.
        }
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        '''
    return query

def otherRefQuery(QID,p):
    query='''PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?ref WHERE {
        wd:'''+QID+''' p:'''+p+''' ?statement.
        ?statement ps:'''+p+''' ?education.
 
    
        OPTIONAL {
        ?statement prov:wasDerivedFrom ?refnode.
        ?refnode pr:P5933 ?ref.
        }

      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
        }
        '''
    return query
    
      
def getData(queryString):
    sparql = SPARQLWrapper("https://query.wikidata.org/bigdata/namespace/wdq/sparql")
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        data = sparql.query().convert()
   # ret is a stream with the results in XML, see <http://www.w3.org/TR/rdf-sparql-XMLres/>
    except :
        print('query failed')

    #url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    #data = requests.get(url, params={'query': query}).json()
    return data['results']['bindings']
    
    #url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    #data = requests.get(url, params={'query': query, 'format': 'json','encoding':'UTF-8'}).json()
    #return data#['results']['bindings']

In [58]:
for index,row in df.iterrows():
    if index%50000==0:
        print(index)
        df.to_csv('mergedPeople.csv')
    if row['queer']==True:
        if len(str(row['P91']))>0:
            l=[]
            try:
                for r in getData(writeRefQuery(row['QID'],'P91')):
                    l.append(r['ref']['value'])
                
            except:
                try:
                    for r in getData(otherRefQuery(row['QID'],'P91')):
                        l.append('https://twitter.com/i/web/status/'+r['ref']['value'])
                except:
                    pass
            df.at[index,'P91ref']=l
                
        if len(str(row['P6553']))>0:
            l=[]
            try:
                for r in getData(writeRefQuery(row['QID'],'P6553')):
                    l.append(r['ref']['value'])
                
            
            except:
                try:
                    for r in getData(otherRefQuery(row['QID'],'P6553')):
                        l.append('https://twitter.com/i/web/status/'+r['ref']['value'])
                except:
                    pass
            df.at[index,'P6553ref']=l
        
        l=[]    
        try:
            for r in getData(writeRefQuery(row['QID'],'P21')):
                l.append(r['ref']['value'])
                
        except:
            try:
                for r in getData(otherRefQuery(row['QID'],'P21')):
                    l.append('https://twitter.com/i/web/status/'+r['ref']['value'])
            except:
                pass
        df.at[index,'P21ref']=l
df.to_csv('mergedPeople.csv')                

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000


In [59]:
df.to_csv('mergedPeople.csv')

In [60]:
df['P91ref'].describe()

count     1464111
unique        695
top              
freq      1463313
Name: P91ref, dtype: object

In [54]:
res=getData(writeRefQuery('Q173399','P91'))
print(res)

[{}]


In [55]:
l=[]
for r in res:
    l.append(r['ref']['type'])
print(l)

KeyError: 'ref'